In [1]:
!pip install google-generativeai langgraph python-dotenv

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.9.0
    Uninstalling google-ai-generativelanguage-0.9.0:
      Successfully uninstalled google-ai-generativelanguage-0.9.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.0.0 requires google-ai-generativelanguage<1.0.0,>=0.7.0, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [5]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import google.generativeai as genai
import os

In [7]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

In [9]:
class ChatState(TypedDict):
    messages: List[str]

In [25]:
def chatbot_node(state: ChatState) -> ChatState:
    model = genai.GenerativeModel("gemini-2.0-flash")
    user_message = state["messages"][-1]
    response = model.generate_content(user_message)
    return {"messages": state["messages"] + [response.text]}


In [27]:
graph = StateGraph(ChatState)
graph.add_node("chatbot", chatbot_node)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

app = graph.compile()

In [29]:
final_state = app.invoke({"messages": ["What do you know about LangGraph?"]})
print("Chatbot:", final_state["messages"][-1])

Chatbot: LangGraph is a Python library built by the LangChain team to help developers build more robust and reliable conversational AI applications. It essentially provides a framework for constructing **stateful, multi-actor AI applications** by representing them as directed graphs.  Think of it as a way to orchestrate complex interactions between different components (like LLMs, tools, and decision-making logic) over multiple steps.

Here's a breakdown of what I know about LangGraph:

**Key Concepts and Features:**

*   **Graph-Based Architecture:** At its core, LangGraph uses a directed graph data structure.  Nodes in the graph represent different actors or components in your application (like a prompt, an LLM, a tool, a decision maker), and edges define the flow of information and control between them.

*   **State Management:**  A central concept is the state of the graph. This state is persisted across multiple rounds of interaction and can be updated by the actors within the gra